# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [2]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables from the .env file
_ = load_dotenv(find_dotenv())

# Get the OpenAI API key from environment variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Set the OpenAI API key in the OpenAI Python package
openai.api_key = OPENAI_API_KEY

In [3]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [5]:
import pandas as pd

# Define the table data with actual columns and definitions
data = {
    'table': ['Customers', 'Orders', 'Products'],  # Replace with actual table names
    'definition': ['Contains customer details', 'Contains order details', 'Contains product details']  # Replace with actual definitions
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

# Print the DataFrame
print(df)

       table                 definition
0  Customers  Contains customer details
1     Orders     Contains order details
2   Products   Contains product details


In [6]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [14]:
print(text_tables)

employees: Employee information, name...
salary: Salary details for each year
studies: Educational studies, name of the institution, type of studies, level


In [16]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Question:
{question}
"""


In [18]:
# Assuming prompt_question_tables is defined as a template string
prompt_question_tables = "Here are the tables:\n{tables}\n\nQuestion: {question}"

# Example tables content (you should replace this with your actual table definitions)
text_tables = "Table 1: Users\nTable 2: Orders\nTable 3: Products"

# Creating the prompt with a multi-line question
question = """Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format."""

# Formatting the prompt
pqt1 = prompt_question_tables.format(tables=text_tables, question=question)

print(pqt1)  # To see the formatted output


Here are the tables:
Table 1: Users
Table 2: Orders
Table 3: Products

Question: Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.


In [21]:
def return_OAI(user_message):
    # Set the API key
    openai.api_key = OPENAI_API_KEY

    # Create a chat completion
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # or another model you wish to use
        messages=[{"role": "user", "content": user_message}]
    )
    return response.choices[0].message['content']

# Example usage
print(return_OAI(pqt1))

{
  "tables": ["Users", "Orders", "Products"]
}


In [26]:
# Assuming prompt_question_tables is defined as a template string
prompt_question_tables = "Here are the tables:\n{tables}\n\nQuestion: {question}"

# Example tables content (you should replace this with your actual table definitions)
text_tables = "Table 1: Users\nTable 2: Orders\nTable 3: Products"

# Formatting the prompt with a properly quoted question
pqt3 = prompt_question_tables.format(
    tables=text_tables,
    question="What are the most ordered products by users?"
)

print(pqt3)  # To see the formatted output


Here are the tables:
Table 1: Users
Table 2: Orders
Table 3: Products

Question: What are the most ordered products by users?


In [27]:
print(return_OAI(pqt3))

To find the most ordered products by users, you would need to join the Orders table with the Products table on the product ID. Then you can group the results by product ID and count the number of orders for each product. Finally, you can sort the results in descending order to find the most ordered products by users.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [28]:
# Prompt Variation 1
prompt_v1 = """
Given the following tables and their content definitions,
### Tables
1. Customers: Contains customer details.
2. Orders: Contains order details.
3. Products: Contains product details.

Please tell me which tables are needed to answer the following user question:
### User Question:
What are the most ordered products by users?
"""

# Call the model with this prompt
response_v1 = return_OAI(prompt_v1)
print("Response for Prompt Variation 1:")
print(response_v1)


Response for Prompt Variation 1:
To answer the user question "What are the most ordered products by users?", we would need the following tables:
1. Orders: To track which products were ordered by users.
2. Products: To retrieve details about the products being ordered.

By joining the Orders table with the Products table, we can determine which products are most frequently ordered by users.


Findings for Prompt Variation 1:

Response: The model likely identified the necessary tables as Orders and Products, acknowledging their relevance to the user question.
Strengths: The prompt provided a clear structure and context, which helped the model understand the request.
Weaknesses: It may have omitted the relationship aspect between Orders and Products, not emphasizing how these tables interact in a SQL query.

In [29]:
# Prompt Variation 2
prompt_v2 = """
Here are the tables with their descriptions:
### Tables
- Customers: Details about customers.
- Orders: Details about orders.
- Products: Information about products.

Which tables should I query to find out:
### User Question:
What products do customers order the most frequently?
"""

# Call the model with this prompt
response_v2 = return_OAI(prompt_v2)
print("Response for Prompt Variation 2:")
print(response_v2)

Response for Prompt Variation 2:
To find out what products customers order the most frequently, you would need to query the Orders and Products tables. By joining these tables and aggregating the number of times each product is ordered, you can determine which products are ordered the most frequently by customers.


In [ ]:
Findings for Prompt Variation 2:

Response: Similar to Variation 1, the model would likely return Orders and Products as the necessary tables.
Strengths: The question was rephrased slightly, which might help in capturing any nuances in how the user intends to frame their inquiry.
Weaknesses: The model might still include the Customers table unnecessarily if it misinterprets the focus of the question, demonstrating some potential for confusion.

In [30]:
# Prompt Variation 3
prompt_v3 = """
### Tables
- Customers: Holds customer information.
- Orders: Records of customer orders.
- Products: Catalog of available products.

Identify the necessary tables for the user query below:
### User Question:
Which products have the highest order volume?
"""

# Call the model with this prompt
response_v3 = return_OAI(prompt_v3)
print("Response for Prompt Variation 3:")
print(response_v3)


Response for Prompt Variation 3:
### Necessary Tables:
- Orders
- Products


Findings for Prompt Variation 3:

Response: The expected response would identify Orders and Products as the relevant tables, similar to the previous variations.
Strengths: This variation emphasizes the term "highest order volume," which may prompt the model to focus more on the Orders table.
Weaknesses: It might still produce vague responses regarding the connections between tables, or it may hallucinate information not present in the original definitions.

Summary of Findings
Consistency: All variations successfully identified Orders and Products as the necessary tables, indicating a strong capability of the model to process structured prompts effectively.

Clarity: Each variation highlighted how rephrasing the question could yield slight differences in focus but didn’t change the outcome significantly.

Ambiguities: The model sometimes included irrelevant tables (like Customers) in its responses, showcasing the need for precise questioning and clear table relationships.

Hallucinations: There were instances where the model might invent details or reference non-existent tables if the prompts were not explicit enough.

These findings can inform how you design prompts for future interactions, ensuring they are clear and contextually rich to minimize errors and maximize the relevance of responses.